# Chloe Wang, Qihan Guan, Michael Harris # 
### BAX-422 Group 5 ###
### March 9, 2021 ### 

In [ ]:
pip install pyzipcode 

  Created wheel for pyzipcode: filename=pyzipcode-3.0.1-py3-none-any.whl size=1932419 sha256=37d9db666e6db5b2b6a71ac09875e78788eff76a6ee3eb7e22c1bc4658a52904
  Stored in directory: c:\users\dvcgu\appdata\local\pip\cache\wheels\f6\01\8e\7424c7e867eb5db7bb9531b09b4d968f23e5ac45ec5e7c3cb3
Successfully built pyzipcode
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import packages
from bs4 import BeautifulSoup, Tag
from urllib.request import Request, urlopen
from urllib.request import urlopen
import requests
import time
import warnings
import requests
import os
import json
import re
import codecs
import urllib.request, urllib.error, urllib.parse
import shutil
from lxml import html
from time import sleep
from itertools import zip_longest
from pyzipcode import ZipCodeDatabase
import pandas as pd
import numpy as np

In [39]:
# Ask Input for Car make.
make = str(input("Enter Make(Press enter directly to omit): ")).lower()

# Ask Input for Model
model = str(input("Enter Model(Press enter directly to omit): ")).lower()

# Ask Input for Minimum Year
min_year = ''
while True: 
  year1 = int(input("Please enter vehicle minimum year: (1981-2021): ")) 
  min_year = str(year1)
  if year1 not in range(1981,2022):
    print("**Please enter a valid year**")
    continue
  else:
    break


# Ask Input for Maximum Year
max_year = ''
while True: 
  year2 = int(input("Please enter vehicle maximum year: (1981-2021): ")) 
  max_year = str(year2)
  if year2 not in range(1981,2022):
    print("**Please enter a valid year**")
    continue
  elif year2 < int(min_year):
    print("**Maximum year cannot be older than minimum year**")
    continue
  else:
    break

# Ask Input for Zip Code
url_geo=''
while True: 
  zipcode = str(input("Please enter your 5-digit Zip Code: ")) 
  zipdb = ZipCodeDatabase()
  try: 
    geoinfo = zipdb[zipcode]
    url_geo = str(geoinfo.city).lower().replace(" ","-") +"-"+str(geoinfo.state).lower()+"-"+str(zipcode)
  except: 
    print("**Please enter a valid zipcode**") 
    continue
  else:
    break
print(url_geo)
# Ask Input for Radius
radius=''
while True: 
  radius = str(input("Please enter your search radius in miles (10/25/50/75/100/200/300/400/500/ 0 for any): ")) 
  if radius not in ['10','25','50','75','100','200','300','400','500','0']:
    print("**Please enter a valid radius**")
    continue
  else:
    break


Enter Make(Press enter directly to omit): toyota
Enter Model(Press enter directly to omit): camry
Please enter vehicle minimum year: (1981-2021): 2019
Please enter vehicle maximum year: (1981-2021): 2020
Please enter your 5-digit Zip Code: 11201
brooklyn-ny-11201
Please enter your search radius in miles (10/25/50/75/100/200/300/400/500/ 0 for any): 50


In [40]:
# Compose the URL based on user input. Set to 100 items per page to miminze get requests
url = str("https://www.autotrader.com/cars-for-sale/all-cars/"+make+"/"+model+"/"+url_geo+"?dma=&searchRadius="+
radius+"&location=&marketExtension=include&startYear="+min_year+"&endYear="+max_year+"&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100")

print(url)

https://www.autotrader.com/cars-for-sale/all-cars/toyota/camry/brooklyn-ny-11201?dma=&searchRadius=50&location=&marketExtension=include&startYear=2019&endYear=2020&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100


### Save all the listed vehicle URLs under the user input criteria

In [41]:
# Scrape All URLs from Search:
header = ({'User-Agent':'Mozilla/5.0'}) 

# Initialize a list to save the item urls
url_list = []

# Loop thorugh all the pages available 
i = 0 # set a intial counter 0
while True:
    URL = url + '&firstRecord=' + str(i*100) #Autotrader uses a multiplier of num_records_per_page to navigate to different pages
    response = requests.get(URL, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')
    pagebuttons = soup.find_all('li', {'class':'active'})
    out_of_bound = True
    
    #Check in page number out-of-bound
    for buttons in pagebuttons:
        if(len(buttons.find_all('a', {'role':'button'}))!=0):
            out_of_bound = False
    
    if out_of_bound == True: # Stop if page number out-of-bound
        break
    #else, scrape url of each listed vehicle for sale
    else:
        items = soup.findAll('div',{"class":"display-flex justify-content-between"})
        for item in items:
            a = item.find("a")
            if not a:
                continue
            url_simplify = 'https://www.autotrader.com' + a['href']
            url_list.append(url_simplify + "\n")
            print(url_simplify)
        #set a time lapse
        time.sleep(4)
        
    #increment counter
    i = i+1
    
  

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=565524822&zip=11201&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D11201%26city%3DBrooklyn%26startYear%3D2019%26numRecords%3D100%26incremental%3Dall%26endYear%3D2020%26modelCodeList%3DCAMRY%26makeCodeList%3DTOYOTA%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DNY%26firstRecord%3D0%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2019&numRecords=100&firstRecord=0&endYear=2020&modelCodeList=CAMRY&makeCodeList=TOYOTA&searchRadius=50&makeCode1=TOYOTA&modelCode1=CAMRY&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=577130906&zip=11201&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D11201%26city%3DBrooklyn%26startYear%3D2019%26numRecords%3D100%26incremental%3Dall%26endYear%3D2020%26modelCodeList%3DCAMRY%26makeCodeList%3DTOYOTA%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=565524822&zip=11201&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D11201%26city%3DBrooklyn%26startYear%3D2019%26numRecords%3D100%26incremental%3Dall%26endYear%3D2020%26modelCodeList%3DCAMRY%26makeCodeList%3DTOYOTA%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DNY%26firstRecord%3D100%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2019&numRecords=100&firstRecord=100&endYear=2020&modelCodeList=CAMRY&makeCodeList=TOYOTA&searchRadius=50&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=578155896&zip=11201&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D11201%26city%3DBrooklyn%26startYear%3D2019%26numRecords%3D100%26incremental%3Dall%26endYear%3D2020%26modelCodeList%3DCAMRY%26makeCodeList%3DTOYOTA%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DNY%26firstRecord%3D

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=565524822&zip=11201&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D11201%26city%3DBrooklyn%26startYear%3D2019%26numRecords%3D100%26incremental%3Dall%26endYear%3D2020%26modelCodeList%3DCAMRY%26makeCodeList%3DTOYOTA%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DNY%26firstRecord%3D200%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2019&numRecords=100&firstRecord=200&endYear=2020&modelCodeList=CAMRY&makeCodeList=TOYOTA&searchRadius=50&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=568771973&zip=11201&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D11201%26city%3DBrooklyn%26startYear%3D2019%26numRecords%3D100%26incremental%3Dall%26endYear%3D2020%26modelCodeList%3DCAMRY%26makeCodeList%3DTOYOTA%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DNY%26firstRecord%3D

In [42]:
#Check the total number of URLs saved
len(url_list)

252

### Open all the URLs and save the listing vehicles' pages in a directory

In [43]:
#Compose a filename based on the user input
file_name = make+'_'+model+'_'+min_year+'_'+max_year+'_'+str(zipcode)+'_'+str(radius)+'miles'

#Write all listed vehicle's url to file_name.txt
with open(os.getcwd()+'/'+file_name+'.txt', 'w+') as filehandle:
    for item_url in url_list:
        filehandle.write('%s' % item_url)

In [44]:
#read the txt file and save the listed vehicles' pages as html files
cars = open(os.getcwd()+'/'+file_name+'.txt', 'r').read().split('\n')
headers = {'User-Agent':'Mozilla/5.0'}  

#create path for 'file_name' folder
path = os.path.join(os.getcwd() , file_name)
if not os.path.exists(path):
    os.makedirs(path)

try:
    #Download each of the pages(URLs) into the folder 'file_name'
    for URL in cars[0:243]:
        listing_id = re.search(r'\?listingId=([0-9]+)&', URL).group(1) #Use listing ID as name for each html file
        page = requests.get(URL, headers=headers) 
        with open(os.path.join(path, str(listing_id)+'.html'), 'w+', encoding="utf-8") as filehandle:
            filehandle.write(page.text)

        #put a 3 sec pause between queries
        time.sleep(3)
except:
    print('Problem with connection...')
        

In [45]:
#Check how many files saved successfully
list = os.listdir(path) 
print(len(list))

231


### Open each saved html file and save the data we want in a pd dataframe
For each listed vehicle, we will save the following information:

Listing ID, Vehicle Condition, Year, Make, Model, Trim Level, List Price, Mileage, Transimission, Drivetrain(FWD/RWD/AWD), Exterior Color, Interior Color, Engine Type, City MPG, Highway Mpg, Seller Type(Private or Dealer), Seller Name, Seller Rating, Seller Number of Reviews, Seller Street Address, Seller City, Seller State, Seller Zipcode, Seller Phone Number

In [46]:
#initialize a data frame to save 

vehicle_info = pd.DataFrame(columns=['Listing ID','Condition', 'Year', 'Make', 'Model','Trim Level','List Price','Mileage',
                                    'Exterior Color', 'Interior Color','Engine Type','Transmission','Drivetrain(FWD/RWD/AWD)', 
                                     'City MPG', 'Highway MPG', 'Seller Type', 'Seller Name','Seller Rating',
                                    'Number of Reviews','Seller Street Address', 'Seller City','Seller State',
                                     'Seller Zipcode','Seller Phone Number'])

for i in range(0, len(list)):
    try:
        page = list[i]
        listing_id = re.search(r'([0-9]+)\.html',list[i]).group(1) #Get Listing ID
        page_file = open(path + '\\' + page, 'r', encoding="utf-8").read() 
        soup = BeautifulSoup(page_file, 'html.parser')

        #selector command for Vehicle Condition
        try:
            title_raw = soup.find('h1', class_ ='text-bold text-size-400 text-size-sm-700').text
            condition = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(1)
        except:
            condition = None
            
        #selector command for Vehicle Year
        try:
            year = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(2)
        except:
            year = None

        #selector command for Vehicle Make
        try:
            make = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(3)
        except:
            make = None
        
        #selector command for Vehicle Model
        try:
            model = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(4)
        except:
            model = None
        
        #selector command for Vehicle Trim Level
        try:
            trim = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+) (.+)',title_raw).group(5)
        except:
            trim = None
        
        #selector command for List Price
        #In most cases, a net price will be displayed
        try:
            price = int(soup.find('span',{'class':'first-price'}).text.replace(',',''))
        except:#For new vehicles, some of them will display only the MSRP as price
            try:
                price_raw = soup.find('span', {'class':'padding-left-2 padding-bottom-1 text-normal second-price'}).find('span', {'class':'text-bold'}).text
                price = int(price_raw.replace(',',''))  
            except:
                price = None
        
        #selector command for Vehicle Mileage
        try:
            #find the element following the mileage icon
            m = soup.find('div', {'style':'background:url(https://www.autotrader.com/content/dam/autotrader/findcar/vdp/specifications/all_spec_icons.svg) -191px -74px;height:25px;width:30px'})
            mileage_raw = re.search(r'(.+) miles',m.parent.find_next_sibling('div').text).group(1)
            mileage = int(mileage_raw.replace(',',''))
        except:
            mileage = None
        
        #selector command for Exterior Color
        try:
            #find the element following the exterior color icon
            excol = soup.find_all('div', {'class':'swatch'})[0]
            extcolor = re.search(r'(.+) Exterior',excol.parent.parent.nextSibling.text).group(1)
        except:
            extcolor = None
        
        #selector command for Interior Color
        try:
            #find the element following the interior color icon
            incol = soup.find_all('div', {'class':'swatch'})[1]
            intcolor = re.search(r'(.+) Interior',incol.parent.parent.nextSibling.text).group(1)
        except:
            intcolor = None
        
        #selector command for Engine Type
        try:
            eng = soup.find('div', {'aria-label':"ENGINE_DESCRIPTION"})
            engine = eng.parent.find_next_sibling('div').text
        except:
            engine = None
        
        #selector command for Transmission
        try:
            trans = soup.find('div', {'aria-label':"TRANSMISSION"})
            transmission = trans.parent.find_next_sibling('div').text
        except:
            transmission = None
        
        #selector command for Drivetrain
        try:
            drive = soup.find('div', {'aria-label':"DRIVE TYPE"})
            drivetrain = drive.parent.find_next_sibling('div').text
        except:
            drivetrain = None
            
        #selector command for City MPG
        try:
            mpg = soup.find('div', {'aria-label':"MPG"}).parent.find_next_sibling('div').text
            city_mpg = re.search(r'([0-9]+) City / ([0-9]+) Highway',mpg).group(1)
        except:
            city_mpg = None
            
        #selector command for Highway MPG
        try:
            mpg = soup.find('div', {'aria-label':"MPG"}).parent.find_next_sibling('div').text
            highway_mpg = re.search(r'([0-9]+) City / ([0-9]+) Highway',mpg).group(2)
        except:
            highway_mpg = None
            
        #Get Seller's type (either private seller or dealer)
        #if find the h2 class for 'private seller' heading, then it's private seller
        if(len(soup.find_all('h2', {'class':'text-bold padding-bottom-3'}))!=0):
            seller_type = 'Private Seller'
        #otherwise, the seller is a dealer    
        else:
            seller_type = 'Dealer'
            
        #Get Seller Name
        if seller_type == 'Dealer':
            try:
                seller_name = soup.find('h2', {'class':"text-left text-bold text-size-20 display-inline"}).text
            except:
                try:
                    seller_name = soup.find('h2', {'class':"text-left text-bold text-size-20"}).text
                except:
                    seller_name = None
        else:
            seller_name = None #Set to 'None' for private sellers because Not Applicable
            
        #Get Seller Rating
        if seller_type == 'Dealer':
            try:
                seller_rating = float(soup.find('span', {'class': "text-bold text-subdued"}).text)
            except:
                seller_rating = None
        else:
            seller_rating = None #Set to 'None' for private sellers because Not Applicable
        
        #Get Number of Reviews for the Seller
        if seller_type == 'Dealer':
            try:
                num_reviews_raw = soup.find('a', {'class': "cursor-pointer padding-left-1"}).text
                num_reviews = int(re.search(r'([0-9]+)', num_reviews_raw).group(1))
            except:
                num_reviews = 0 
        else:
            num_reviews = None #Set to 'None' for private sellers because Not Applicable
        
        #Get Seller Street Address
        addrs = soup.find_all('div', {'data-cmp': "address"})
        address = ''
        for addr in addrs:
            for line in addr:
                address = address + line.text
        try:
            seller_street = re.search(r'([0-9]+ [A-Z].+), [A-Za-z\s]+, ', address).group(1)
        except:
            seller_street = None
            
        #Get Seller City
        try:
            seller_city = re.search(r'(.*,)?\s?(.*), [A-Z]{2} [0-9]{5}', address).group(2)
        except:
            seller_city = None  
            
        #Get Seller State
        try:
            seller_state = re.search(r', ([A-Z]{2}) ([0-9]{5})', address).group(1)
        except:
            seller_state = None
        
        #Get Seller Zipcode
        try:
            seller_zip = re.search(r', ([A-Z]{2}) ([0-9]{5})', address).group(2)
        except:
            seller_zip = None
        
        #Get Seller Phone Number
        try:
            seller_phone = soup.find('span', {'data-cmp':'phoneNumber'}).text
        except:
            seller_phone = None
            
        #save this iteration's result to the i-th row of the dataframe
        vehicle_info.loc[i] = [listing_id, condition, year, make, model, trim, price, mileage, extcolor, intcolor, engine, transmission, drivetrain, city_mpg, 
                       highway_mpg, seller_type, seller_name, seller_rating, num_reviews, seller_street, seller_city, seller_state,
                       seller_zip, seller_phone]

           
    except:
        print('Error with: ' + page)

### View Saved Data

In [47]:
vehicle_info

,Listing ID,Condition,Year,Make,Model,Trim Level,List Price,Mileage,Exterior Color,Interior Color,...,Highway MPG,Seller Type,Seller Name,Seller Rating,Number of Reviews,Seller Street Address,Seller City,Seller State,Seller Zipcode,Seller Phone Number
0,529451459,New,2020,Toyota,Camry,LE,27498,14,Midnight Black Metallic,Black,...,41,Dealer,Toyota Universe,4.5,5128,1485 Route 46 East,Little Falls,NJ,07424,(973) 339-3360
1,534942415,New,2020,Toyota,Camry,LE,27970,0,Midngt Blk Met,Ash Fabric,...,53,Dealer,Toyota of Hackensack,4.1,1281,278 RIVER ST,HACKENSACK,NJ,07601,(201) 334-0784
2,542049889,New,2020,Toyota,Camry,LE Hybrid,29773,6,Midnight Black Metallic,Black,...,41,Dealer,Crestmont Toyota Volkswagen,4.5,519,730 Route 23 North,Pompton Plains,NJ,07444,(862) 228-6262
3,547035412,New,2020,Toyota,Camry,XLE Hybrid,34516,122,Blue,None,...,47,Dealer,Toyota of Morristown,4.6,872,169 Ridgedale Avenue,Morristown,NJ,07960,(862) 325-2051
4,547784004,New,2020,Toyota,Camry,XLE,31963,0,Black,None,...,47,Dealer,Atlantic Toyota,4.5,1699,193 Sunrise Hwy,West Islip,NY,11795,(631) 517-4896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,579352698,Used,2019,Toyota,Camry,XSE w/ Navigation Package,24999,21565,Midnight Black Metallic,Lb30/Cockpit Red,...,41,Dealer,Galaxy Toyota,4.6,964,750 Route 36 East,Eatontown,NJ,07724,None
227,579412030,Used,2019,Toyota,Camry,SE,23938,29471,Wh,None,...,41,Dealer,Westbury Toyota,4.4,1727,1121 OLD COUNTRY RD,WESTBURY,NY,11590,None
228,579469466,Used,2019,Toyota,Camry,SE,20990,10946,Red,Black,...,41,Dealer,"Vroom - Get It Delivered Nationwide, Contact-Free",None,0,None,None,None,None,(646) 952-7515
229,579472294,Used,2020,Toyota,Camry,XSE AWD,29780,7529,Predawn Gray Mica,Black,...,None,Dealer,"Vroom - Get It Delivered Nationwide, Contact-Free",None,0,None,None,None,None,(332) 877-3681


### Save Data to MongoDB

In [21]:
#Connect to MongoDB
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

db = client[file_name] #use the user input criteria to compose a database name
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Toyota_Tacoma_2019_2020_11201_50miles')

Save data from dataframe to MongoDB collection row by row:

In [22]:
#Show database name
print(file_name)
mycollection = db[file_name]
print(mycollection)

Toyota_Tacoma_2019_2020_11201_50miles
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Toyota_Tacoma_2019_2020_11201_50miles'), 'Toyota_Tacoma_2019_2020_11201_50miles')


In [23]:
for i in vehicle_info.index:
    #If seller is a dealer, insert all info
    if vehicle_info['Seller Type'][i] == 'Dealer':
        listing_id = int(vehicle_info['Listing ID'][i])
        try:
            Condition = str(vehicle_info['Condition'][i])
        except:
            Condition = None
        try:
            Year = int(vehicle_info['Year'][i])
        except:
            Year = None
        try:
            Make = str(vehicle_info['Make'][i])
        except:
            Make = None
        try:
            Model = str(vehicle_info['Model'][i])
        except:
            Model = None
        try:
            TrimLevel = str(vehicle_info['Trim Level'][i])
        except:
            TrimLevel = None
        try:
            ListPrice = int(vehicle_info['List Price'][i])
        except:
            ListPrice = None
        try:
            Mileage = int(vehicle_info['Mileage'][i])
        except:
            Mileage = None
        try:
            ExteriorColor = str(vehicle_info['Exterior Color'][i])
        except:
            ExteriorColor = None
        try:
            InteriorColor = str(vehicle_info['Interior Color'][i])
        except:
            InteriorColor = None
        try:
            EngineType = str(vehicle_info['Engine Type'][i])
        except:
            EngineType = None
        try:
            Transmission = str(vehicle_info['Transmission'][i])
        except:
            Transmission = None
        try:
            Drivetrain = str(vehicle_info['Drivetrain(FWD/RWD/AWD)'][i])
        except:
            Drivetrain = None
        try:
            CityMPG = int(vehicle_info['City MPG'][i])
        except:
            CityMPG = None
        try:
            HighwayMPG = int(vehicle_info['Highway MPG'][i])
        except:
            HighwayMPG = None
        SellerType = str(vehicle_info['Seller Type'][i])
        try:
            SellerName = str(vehicle_info['Seller Name'][i])
        except:
            SellerName = None
        try:
            SellerRating = float(vehicle_info['Seller Rating'][i])
        except:
            SellerRating = None
        try:
            NumberofReviews = int(vehicle_info['Number of Reviews'][i])
        except:
            NumberofReviews = None
        try:
            SellerStreetAddress = str(vehicle_info['Seller Street Address'][i])
        except:
            SellerStreetAddress = None
        try:
            SellerCity = str(vehicle_info['Seller City'][i])
        except:
            SellerCity = None
        try:
            SellerState = str(vehicle_info['Seller State'][i])
        except:
            SellerState = None
        try:
            SellerZipcode = str(vehicle_info['Seller Zipcode'][i])
        except:
            SellerZipcode = None
        try:
            SellerPhoneNumber = str(vehicle_info['Seller Phone Number'][i])
        except:
            SellerPhoneNumber = None
        
        mydict = {
            '_id' : listing_id,
            'Condition': Condition,
            'Year': Year,
            'Make': Make,
            'Model': Model,
            'Trim Level': TrimLevel,
            'List Price': ListPrice,
            'Mileage': Mileage,
            'Exterior Color': ExteriorColor,
            'Interior Color': InteriorColor,
            'Engine Type': EngineType,
            'Transmission': Transmission,
            'Drivetrain': Drivetrain,
            'City MPG': CityMPG,
            'Highway MPG': HighwayMPG,
            'Seller Type': SellerType,
            'Seller Name': SellerName,
            'Seller Rating': SellerRating,
            'Number of Reviews': NumberofReviews,
            'Seller Street Address': SellerStreetAddress,
            'Seller City': SellerCity,
            'Seller State': SellerState,
            'Seller Zipcode': SellerZipcode,
            'Seller Phone Number': SellerPhoneNumber 
        }
        
        #insert this listed vehicle's data to MongoDB database 
        mycollection.insert_one(mydict)
    
    #If seller is a private seller, do not save Seller Name, Seller Rating or Number of Reviews
    else:
        listing_id = int(vehicle_info['Listing ID'][i])
        try:
            Condition = str(vehicle_info['Condition'][i])
        except:
            Condition = None
        try:
            Year = int(vehicle_info['Year'][i])
        except:
            Year = None
        try:
            Make = str(vehicle_info['Make'][i])
        except:
            Make = None
        try:
            Model = str(vehicle_info['Model'][i])
        except:
            Model = None
        try:
            TrimLevel = str(vehicle_info['Trim Level'][i])
        except:
            TrimLevel = None
        try:
            ListPrice = int(vehicle_info['List Price'][i])
        except:
            ListPrice = None
        try:
            Mileage = int(vehicle_info['Mileage'][i])
        except:
            Mileage = None
        try:
            ExteriorColor = str(vehicle_info['Exterior Color'][i])
        except:
            ExteriorColor = None
        try:
            InteriorColor = str(vehicle_info['Interior Color'][i])
        except:
            InteriorColor = None
        try:
            EngineType = str(vehicle_info['Engine Type'][i])
        except:
            EngineType = None
        try:
            Transmission = str(vehicle_info['Transmission'][i])
        except:
            Transmission = None
        try:
            Drivetrain = str(vehicle_info['Drivetrain(FWD/RWD/AWD)'][i])
        except:
            Drivetrain = None
        try:
            CityMPG = int(vehicle_info['City MPG'][i])
        except:
            CityMPG = None
        try:
            HighwayMPG = int(vehicle_info['Highway MPG'][i])
        except:
            HighwayMPG = None
        SellerType = str(vehicle_info['Seller Type'][i])
        try:
            SellerStreetAddress = str(vehicle_info['Seller Street Address'][i])
        except:
            SellerStreetAddress = None
        try:
            SellerCity = str(vehicle_info['Seller City'][i])
        except:
            SellerCity = None
        try:
            SellerState = str(vehicle_info['Seller State'][i])
        except:
            SellerState = None
        try:
            SellerZipcode = str(vehicle_info['Seller Zipcode'][i])
        except:
            SellerZipcode = None
        try:
            SellerPhoneNumber = str(vehicle_info['Seller Phone Number'][i])
        except:
            SellerPhoneNumber = None
        
        mydict = {
            '_id' : listing_id,
            'Condition': Condition,
            'Year': Year,
            'Make': Make,
            'Model': Model,
            'Trim Level': TrimLevel,
            'List Price': ListPrice,
            'Mileage': Mileage,
            'Exterior Color': ExteriorColor,
            'Interior Color': InteriorColor,
            'Engine Type': EngineType,
            'Transmission': Transmission,
            'Drivetrain': Drivetrain,
            'City MPG': CityMPG,
            'Highway MPG': HighwayMPG,
            'Seller Type': SellerType,
            'Seller Street Address': SellerStreetAddress,
            'Seller City': SellerCity,
            'Seller State': SellerState,
            'Seller Zipcode': SellerZipcode,
            'Seller Phone Number': SellerPhoneNumber 
        }
        
        #insert this listed vehicle's data to MongoDB database 
        mycollection.insert_one(mydict)
        
    print('Row '+str(i)+' successfully inserted!\n')
        

Row 0 successfully inserted!

Row 1 successfully inserted!

Row 2 successfully inserted!

Row 3 successfully inserted!

Row 4 successfully inserted!

Row 5 successfully inserted!

Row 6 successfully inserted!

Row 7 successfully inserted!

Row 8 successfully inserted!

Row 9 successfully inserted!

Row 10 successfully inserted!

Row 11 successfully inserted!

Row 12 successfully inserted!

Row 13 successfully inserted!

Row 14 successfully inserted!

Row 15 successfully inserted!

Row 16 successfully inserted!

Row 17 successfully inserted!

Row 18 successfully inserted!

Row 19 successfully inserted!

Row 20 successfully inserted!

Row 21 successfully inserted!

Row 22 successfully inserted!

Row 23 successfully inserted!

Row 24 successfully inserted!

Row 25 successfully inserted!

Row 26 successfully inserted!

Row 27 successfully inserted!

Row 28 successfully inserted!

Row 29 successfully inserted!

Row 30 successfully inserted!

Row 31 successfully inserted!

Row 32 successfull

### Run Summary Stats to find meaningful insights

Find this vehicle's market average price by condition(New/Used/Certified Used)

In [24]:
file_name #print database name composed by user input

'Toyota_Tacoma_2019_2020_11201_50miles'

In [25]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Condition",
            "avgPrice": {"$avg": "$List Price"}
        }
    }
])

for r in result:
    print(r)

{'_id': 'Certified', 'avgPrice': 26344.8}
{'_id': 'Used', 'avgPrice': 28131.5}


Find this vehicle's average listing price by Seller Type

In [26]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Seller Type",
            "Average Price": {"$avg": "$List Price"}
        }
    }
])

for r in result:
    print(r)

{'_id': 'Dealer', 'Average Price': 27902.4358974359}


Find how many dealers and how many private sellers who sell this vehicle

In [27]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Seller Type",
            "Count": {"$sum": 1}
        }
    }
])

for r in result:
    print(r)

{'_id': 'Dealer', 'Count': 39}


Find this vehicle's market average price by exterior color (Limit to Top 10 by price)

In [28]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Exterior Color",
            "Average Price": {"$avg": "$List Price"}
        }
    },
    { "$sort": { "Average Price": -1 } },
    { "$limit": 10 }
])

for r in result:
    print(r)

{'_id': 'Blue', 'Average Price': 41162.0}
{'_id': 'Silver', 'Average Price': 38330.0}
{'_id': 'Super White', 'Average Price': 37866.0}
{'_id': 'Blizzard Pearl', 'Average Price': 37770.0}
{'_id': 'Midnight Black Metallic', 'Average Price': 36395.5}
{'_id': 'Alumina Jade Metallic', 'Average Price': 35495.0}
{'_id': 'Barcelona Red Metallic', 'Average Price': 32800.0}
{'_id': 'White', 'Average Price': 32492.5}
{'_id': 'Blueprint', 'Average Price': 30977.0}
{'_id': 'Celestial Silver Metallic', 'Average Price': 29403.333333333332}


Find this vehicle's market average price by trim level

In [29]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Trim Level",
            "Average Price": {"$avg": "$List Price"}
        }
    },
    { "$sort": { "Average Price": -1 } }
])

for r in result:
    print(r)

{'_id': 'TRD Pro', 'Average Price': 56244.5}
{'_id': '4WD TRD-Off Road Premium', 'Average Price': 41162.0}
{'_id': 'Limited Platinum', 'Average Price': 39447.5}
{'_id': 'Limited', 'Average Price': 38522.5}
{'_id': 'TRD Sport', 'Average Price': 38330.0}
{'_id': 'TRD Off-Road', 'Average Price': 33891.0}
{'_id': 'SR', 'Average Price': 32800.0}
{'_id': 'AWD Limited', 'Average Price': 30977.0}
{'_id': 'XLE', 'Average Price': 30240.0}
{'_id': 'FWD LE', 'Average Price': 27730.0}
{'_id': 'XSE', 'Average Price': 26907.0}
{'_id': 'SE', 'Average Price': 26485.0}
{'_id': 'AWD LE', 'Average Price': 26233.0}
{'_id': 'AWD V6', 'Average Price': 25980.0}
{'_id': 'AWD XLE', 'Average Price': 25088.0}
{'_id': 'L', 'Average Price': 24990.0}
{'_id': 'None', 'Average Price': 21942.666666666668}
{'_id': 'LE AWD', 'Average Price': 21888.0}
{'_id': 'LE Sedan w/ Premium Package', 'Average Price': 20495.0}
{'_id': 'XSE Sedan', 'Average Price': 18888.0}
{'_id': 'LE', 'Average Price': 18624.333333333332}
{'_id': 'L

Find this vehicle's market average price first by condition and second by trim level

In [30]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": 
                {"condition":"$Condition",
                 "trim":"$Trim Level"},
            "Average Price": {"$avg": "$List Price"}
        }
    },
    { "$sort": { "Average Price": -1 } }
])

for r in result:
    print(r)

{'_id': {'condition': 'Used', 'trim': 'TRD Pro'}, 'Average Price': 56244.5}
{'_id': {'condition': 'Certified', 'trim': '4WD TRD-Off Road Premium'}, 'Average Price': 41162.0}
{'_id': {'condition': 'Used', 'trim': 'Limited Platinum'}, 'Average Price': 39447.5}
{'_id': {'condition': 'Used', 'trim': 'Limited'}, 'Average Price': 38522.5}
{'_id': {'condition': 'Used', 'trim': 'TRD Sport'}, 'Average Price': 38330.0}
{'_id': {'condition': 'Used', 'trim': 'TRD Off-Road'}, 'Average Price': 33891.0}
{'_id': {'condition': 'Used', 'trim': 'SR'}, 'Average Price': 32800.0}
{'_id': {'condition': 'Used', 'trim': 'AWD Limited'}, 'Average Price': 30977.0}
{'_id': {'condition': 'Used', 'trim': 'XLE'}, 'Average Price': 30240.0}
{'_id': {'condition': 'Used', 'trim': 'FWD LE'}, 'Average Price': 27730.0}
{'_id': {'condition': 'Used', 'trim': 'XSE'}, 'Average Price': 27195.0}
{'_id': {'condition': 'Used', 'trim': 'SE'}, 'Average Price': 26485.0}
{'_id': {'condition': 'Certified', 'trim': 'XSE'}, 'Average Price

Find what exterior colors are popular in the market

In [35]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Exterior Color",
            "Count": {"$sum": 1}
        }
    },
    { "$sort": { "Count": -1 } }
])

for r in result:
    print(r)

{'_id': 'Super White', 'Count': 4}
{'_id': 'Celestial Silver Metallic', 'Count': 3}
{'_id': 'Ruby Flare Pearl', 'Count': 2}
{'_id': 'Midnight Black Metallic', 'Count': 2}
{'_id': 'Classic Silver Metallic', 'Count': 2}
{'_id': 'White', 'Count': 2}
{'_id': 'Predawn Gray Mica', 'Count': 2}
{'_id': 'Celestite', 'Count': 2}
{'_id': 'Blizzard Pearl', 'Count': 2}
{'_id': 'Magnetic Gray Metallic', 'Count': 2}
{'_id': 'Wind Chill Pearl', 'Count': 2}
{'_id': 'Black Sand Pearl', 'Count': 1}
{'_id': 'Blueprint', 'Count': 1}
{'_id': 'Silver Sky Metallic', 'Count': 1}
{'_id': 'Barcelona Red Metallic', 'Count': 1}
{'_id': 'Red', 'Count': 1}
{'_id': 'Blue Streak Metallic', 'Count': 1}
{'_id': 'Gray', 'Count': 1}
{'_id': 'Alumina Jade Metallic', 'Count': 1}
{'_id': 'Supersonic Red', 'Count': 1}
{'_id': 'Silver', 'Count': 1}
{'_id': 'Silver Knockout Metallic', 'Count': 1}
{'_id': 'None', 'Count': 1}
{'_id': 'Blue', 'Count': 1}
{'_id': 'Celestial', 'Count': 1}


Find this vehicle's average listing price from dealers that have ratings higher than 4.0

In [36]:
result = mycollection.aggregate([
    { "$match": { "Seller Rating": { "$gte": 4.0} } },
    {
        "$group": {
            "_id": "4.0+",
            "Average Price": {"$avg": "$List Price"}
        }
    },
    { "$sort": { "Average Price": -1 } }
])

for r in result:
    print(r)


{'_id': '4.0+', 'Average Price': 27775.470588235294}


Find dealer's average rating

In [37]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "Dealer Rating",
            "Average Rating": {"$avg": "$Seller Rating"}
        }
    },
    { "$sort": { "Average Rating": -1 } }
])

for r in result:
    print(r)

{'_id': 'Dealer Rating', 'Average Rating': 4.477142857142857}


Find the seller contact information who has lowest listing price

In [38]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": {
                "Seller Name":"$Seller Name",
                "Seller Phone Number":"$Seller Phone Number"
            },
            "Min Price": {"$min": "$List Price"}
        }
    },
    { "$sort": { "Min Price": 1 } }
])

for r in result:
    print(r)
    break

{'_id': {'Seller Name': 'Advantage Toyota', 'Seller Phone Number': '(516) 218-3657'}, 'Min Price': 16098}
